<a href="https://colab.research.google.com/github/Tiwari666/pyspark/blob/main/pyspark_imputation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
import pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=7316852cb9ea8e194c06728393009d04b9ae09c2a1953c4266f61bc28b3606e3
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [22]:
from pyspark.sql import SparkSession
sparkSession=SparkSession.builder.appName('Practise_Session').getOrCreate()

In [62]:
df_pyspark=sparkSession.read.csv('test2.csv',header=True,inferSchema=True)
df_pyspark.show()

+-----+----+------+----------+
| Name| Age|Salary|Experience|
+-----+----+------+----------+
|Peter|  58| 13000|         9|
| Jack|  52| 10000|         3|
| John|  37|  7000|         5|
|  Bob|  54|  5000|         8|
| NULL|NULL|  NULL|      NULL|
|Katie|  52|  7000|         2|
|Katie|  30|  5000|         3|
| Rose|  39| 14000|         2|
| Jack|  40|  3000|         4|
| Rose|  56|  1000|         1|
| Emma|  44|  3000|         8|
| NULL|NULL|  NULL|      NULL|
|Frank|  24| 15000|         4|
|David|  27| 12000|         2|
| Noah|  53|  2000|         3|
| Rose|  22| 14000|         3|
|  Sam|  39|  1000|        10|
| Rose|  28|  3000|         8|
| Noah|  54|  6000|         5|
|Peter|  59| 15000|         5|
+-----+----+------+----------+



In [63]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Experience: integer (nullable = true)



In [25]:
#drop the columns
df_pyspark  = df_pyspark.drop('Name')
df_pyspark.show()

+----+------+----------+
| Age|Salary|Experience|
+----+------+----------+
|  58| 13000|         9|
|  52| 10000|         3|
|  37|  7000|         5|
|  54|  5000|         8|
|NULL|  NULL|      NULL|
|  52|  7000|         2|
|  30|  5000|         3|
|  39| 14000|         2|
|  40|  3000|         4|
|  56|  1000|         1|
|  44|  3000|         8|
|NULL|  NULL|      NULL|
|  24| 15000|         4|
|  27| 12000|         2|
|  53|  2000|         3|
|  22| 14000|         3|
|  39|  1000|        10|
|  28|  3000|         8|
|  54|  6000|         5|
|  59| 15000|         5|
+----+------+----------+



In [26]:
df_pyspark.show()

+----+------+----------+
| Age|Salary|Experience|
+----+------+----------+
|  58| 13000|         9|
|  52| 10000|         3|
|  37|  7000|         5|
|  54|  5000|         8|
|NULL|  NULL|      NULL|
|  52|  7000|         2|
|  30|  5000|         3|
|  39| 14000|         2|
|  40|  3000|         4|
|  56|  1000|         1|
|  44|  3000|         8|
|NULL|  NULL|      NULL|
|  24| 15000|         4|
|  27| 12000|         2|
|  53|  2000|         3|
|  22| 14000|         3|
|  39|  1000|        10|
|  28|  3000|         8|
|  54|  6000|         5|
|  59| 15000|         5|
+----+------+----------+



In [ ]:
# df_pyspark.na.drop().show()

In [35]:
from pyspark.sql.functions import mean

# Impute missing values in the 'Age', 'Salary', and 'ExtraColumn' columns with their respective mean values
mean_values = df_pyspark.select([mean('Age').alias('mean_age'),
                                 mean('Salary').alias('mean_salary'),
                                 mean('Experience').alias('mean_Experience')]).collect()

# Extract mean values
mean_age = mean_values[0]['mean_age']
mean_salary = mean_values[0]['mean_salary']
mean_Experience= mean_values[0]['mean_Experience']

# Fill missing values with mean values
imputed_df = df_pyspark.fillna({'Age': mean_age, 'Salary': mean_salary, 'Experience': mean_Experience})

# Show the DataFrame with imputed values
imputed_df.show()

+---+------+----------+
|Age|Salary|Experience|
+---+------+----------+
| 58| 13000|         9|
| 52| 10000|         3|
| 37|  7000|         5|
| 54|  5000|         8|
| 42|  7555|         4|
| 52|  7000|         2|
| 30|  5000|         3|
| 39| 14000|         2|
| 40|  3000|         4|
| 56|  1000|         1|
| 44|  3000|         8|
| 42|  7555|         4|
| 24| 15000|         4|
| 27| 12000|         2|
| 53|  2000|         3|
| 22| 14000|         3|
| 39|  1000|        10|
| 28|  3000|         8|
| 54|  6000|         5|
| 59| 15000|         5|
+---+------+----------+



In [39]:
mean_values_list = [mean_age, mean_salary, mean_Experience]
mean_values_list

[42.666666666666664, 7555.555555555556, 4.722222222222222]

In [37]:
df_pyspark.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Experience: integer (nullable = true)



In [44]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['Age', 'Experience', 'Salary'],
    outputCols=["{}_imputed".format(c) for c in ['Age', 'Experience', 'Salary']]
    ).setStrategy("median")

In [45]:
# Add imputation cols to df
imputer.fit(df_pyspark).transform(df_pyspark).show()

+----+------+----------+-----------+------------------+--------------+
| Age|Salary|Experience|Age_imputed|Experience_imputed|Salary_imputed|
+----+------+----------+-----------+------------------+--------------+
|  58| 13000|         9|         58|                 9|         13000|
|  52| 10000|         3|         52|                 3|         10000|
|  37|  7000|         5|         37|                 5|          7000|
|  54|  5000|         8|         54|                 8|          5000|
|NULL|  NULL|      NULL|         40|                 4|          6000|
|  52|  7000|         2|         52|                 2|          7000|
|  30|  5000|         3|         30|                 3|          5000|
|  39| 14000|         2|         39|                 2|         14000|
|  40|  3000|         4|         40|                 4|          3000|
|  56|  1000|         1|         56|                 1|          1000|
|  44|  3000|         8|         44|                 8|          3000|
|NULL|

In [68]:
from pyspark.ml.feature import Imputer

# Define the Imputer transformation with the original column names
imputer = Imputer(
    inputCols=['Age', 'Experience', 'Salary'],
    outputCols=['Age', 'Experience', 'Salary']  # Use the original column names
).setStrategy("median")

# Fit the Imputer model and transform the DataFrame
imputed_df = imputer.fit(df_pyspark).transform(df_pyspark)

# Show the DataFrame with imputed values and original column names
imputed_df.show()

+-----+---+------+----------+
| Name|Age|Salary|Experience|
+-----+---+------+----------+
|Peter| 58| 13000|         9|
| Jack| 52| 10000|         3|
| John| 37|  7000|         5|
|  Bob| 54|  5000|         8|
| NULL| 40|  6000|         4|
|Katie| 52|  7000|         2|
|Katie| 30|  5000|         3|
| Rose| 39| 14000|         2|
| Jack| 40|  3000|         4|
| Rose| 56|  1000|         1|
| Emma| 44|  3000|         8|
| NULL| 40|  6000|         4|
|Frank| 24| 15000|         4|
|David| 27| 12000|         2|
| Noah| 53|  2000|         3|
| Rose| 22| 14000|         3|
|  Sam| 39|  1000|        10|
| Rose| 28|  3000|         8|
| Noah| 54|  6000|         5|
|Peter| 59| 15000|         5|
+-----+---+------+----------+



# **Pyspark Filteration:**

In [70]:
# Salary of the people less than or equal to 20000
imputed_df.filter("Salary<=15000").show()

+-----+---+------+----------+
| Name|Age|Salary|Experience|
+-----+---+------+----------+
|Peter| 58| 13000|         9|
| Jack| 52| 10000|         3|
| John| 37|  7000|         5|
|  Bob| 54|  5000|         8|
| NULL| 40|  6000|         4|
|Katie| 52|  7000|         2|
|Katie| 30|  5000|         3|
| Rose| 39| 14000|         2|
| Jack| 40|  3000|         4|
| Rose| 56|  1000|         1|
| Emma| 44|  3000|         8|
| NULL| 40|  6000|         4|
|Frank| 24| 15000|         4|
|David| 27| 12000|         2|
| Noah| 53|  2000|         3|
| Rose| 22| 14000|         3|
|  Sam| 39|  1000|        10|
| Rose| 28|  3000|         8|
| Noah| 54|  6000|         5|
|Peter| 59| 15000|         5|
+-----+---+------+----------+



In [71]:
imputed_df.filter("Salary>10000").select(['Salary','Age']).show()

+------+---+
|Salary|Age|
+------+---+
| 13000| 58|
| 14000| 39|
| 15000| 24|
| 12000| 27|
| 14000| 22|
| 15000| 59|
+------+---+



Display of rows either the salary is less than or equal to 20000, or the salary is greater than or equal to 15000:

In [72]:
# Filter rows where 'Salary' satisfies either of the conditions, and show the result
df_pyspark.filter((df_pyspark['Salary']<=15000) | (df_pyspark['Salary']>=10000)).show()

+-----+---+------+----------+
| Name|Age|Salary|Experience|
+-----+---+------+----------+
|Peter| 58| 13000|         9|
| Jack| 52| 10000|         3|
| John| 37|  7000|         5|
|  Bob| 54|  5000|         8|
|Katie| 52|  7000|         2|
|Katie| 30|  5000|         3|
| Rose| 39| 14000|         2|
| Jack| 40|  3000|         4|
| Rose| 56|  1000|         1|
| Emma| 44|  3000|         8|
|Frank| 24| 15000|         4|
|David| 27| 12000|         2|
| Noah| 53|  2000|         3|
| Rose| 22| 14000|         3|
|  Sam| 39|  1000|        10|
| Rose| 28|  3000|         8|
| Noah| 54|  6000|         5|
|Peter| 59| 15000|         5|
+-----+---+------+----------+



In [73]:
# Filter rows where 'Salary' is not less than or equal to 20000, and show the result
df_pyspark.filter(~(df_pyspark['Salary']<=10000)).show()

+-----+---+------+----------+
| Name|Age|Salary|Experience|
+-----+---+------+----------+
|Peter| 58| 13000|         9|
| Rose| 39| 14000|         2|
|Frank| 24| 15000|         4|
|David| 27| 12000|         2|
| Rose| 22| 14000|         3|
|Peter| 59| 15000|         5|
+-----+---+------+----------+



# **Pyspark GroupBy And Aggregate Functions:**

In [75]:
from pyspark.sql import functions as F

# Group by 'Name' and aggregate to find the maximum salary for each group
max_salary_df = imputed_df.groupBy('Age').agg(F.max('Salary').alias('Max'))

# Show the DataFrame with the maximum salary for each group
max_salary_df.show()

+---+-----+
|Age|  Max|
+---+-----+
| 53| 2000|
| 28| 3000|
| 27|12000|
| 44| 3000|
| 22|14000|
| 52|10000|
| 40| 6000|
| 54| 6000|
| 37| 7000|
| 59|15000|
| 39|14000|
| 24|15000|
| 56| 1000|
| 58|13000|
| 30| 5000|
+---+-----+



In [77]:
df_pyspark.groupBy('Name').count().show()

+-----+-----+
| Name|count|
+-----+-----+
| Rose|    4|
| Jack|    2|
| NULL|    2|
|Katie|    2|
| Noah|    2|
|  Bob|    1|
| John|    1|
|  Sam|    1|
| Emma|    1|
|Peter|    2|
|David|    1|
|Frank|    1|
+-----+-----+



In [78]:
# Apply aggregate functions
df_pyspark.agg(
    F.avg('Age').alias('Average_Age'),
    F.sum('Salary').alias('Total_Salary'),
    F.max('Experience').alias('Max_Experience'),
    F.min('Age').alias('Min_Age'),
    F.count('Name').alias('Total_Count'),
    F.countDistinct('Name').alias('Distinct_Name_Count'),
    F.stddev('Salary').alias('Salary_StdDev'),
    F.variance('Salary').alias('Salary_Variance')
).show()

+------------------+------------+--------------+-------+-----------+-------------------+-----------------+------------------+
|       Average_Age|Total_Salary|Max_Experience|Min_Age|Total_Count|Distinct_Name_Count|    Salary_StdDev|   Salary_Variance|
+------------------+------------+--------------+-------+-----------+-------------------+-----------------+------------------+
|42.666666666666664|      136000|            10|     22|         18|                 11|5113.099925649136|2.61437908496732E7|
+------------------+------------+--------------+-------+-----------+-------------------+-----------------+------------------+

